In [ ]:
from skimage import data, transform, io
from skimage.color import rgb2gray
import numpy as np
import pandas as pd
import napari
from napari.utils.transforms import Affine, CompositeAffine
import matplotlib.pyplot as plt
from MILWRM.MxIF import *

---
# Transform all marker images into ST space and save to TIFFs

In [ ]:
def transform_tiffs(
    affine,
    shape,
    tiffdir,
    read_suffix="_stitched_downsample8.tif",
    save_suffix="_stitched_downsample8_6723_4.tif",
    read_mask="WD86055_TISSUE_MASK_stitched_downsample8.tif",
    save_mask="WD86055_TISSUE_MASK_stitched_downsampled8_6723_4.tif",
    flip=False,
):
    # individual markers first
    for marker in eliotmarkers:
        print(marker, end=": ")
        tmp = io.imread("{}/{}{}".format(tiffdir, marker, read_suffix))
        if flip:
            # flip horizontally the MxIF image
            tmp = tmp[::-1,::-1]
        image = transform.warp(tmp, transform.AffineTransform(affine), output_shape = abs(shape))
        # downsample image
        #image = block_reduce(image, block_size=(8, 8), func=np.mean, cval=0)
        print(image.shape)
        io.imsave(arr=image, fname="{}/{}{}".format(tiffdir, marker, save_suffix))
    # tissue mask if needed
    if read_mask is not None and save_mask is not None:
        print("tissue mask", end=": ")
        tmp = io.imread("{}/{}".format(tiffdir, read_mask))
        if flip:
            # flip horizontally the MxIF image
            tmp = tmp[::-1,::-1]
        image = transform.warp(tmp, transform.AffineTransform(affine), output_shape = abs(shape))
        print(image.shape)
        io.imsave(arr=image, fname="{}/{}".format(tiffdir, save_mask))

In [ ]:
def transform_imgs(
    affine,
    shape,
    img_in,
    img_out,
    flip=False,
    downsample_fact=None,
    plot=True,
):
    a = img.from_npz(img_in)
    shape[2] = a.n_ch  # replace third dim of shape with number of channels in img
    if flip:
        a.img = transform.warp(a.img[::-1,::-1,:], transform.AffineTransform(affine), output_shape=shape)
        a.mask = transform.warp(a.mask[::-1,::-1], transform.AffineTransform(affine), output_shape=shape[:2])
    else:
        a.img = transform.warp(a.img, transform.AffineTransform(affine), output_shape=shape)
        a.mask = transform.warp(a.mask, transform.AffineTransform(affine), output_shape=shape[:2])
    if downsample_fact is not None:
        a.downsample(fact=downsample_fact)
        
    if plot:
        rgb = a.show(channels=["COLLAGEN","VIMENTIN","BCATENIN"], RGB=True)
    a.to_npz(img_out)

In [ ]:
blocks = [
    #"WD86056",
    #"WD86056",
    #"WD86056",
    #"WD86055",
    #"WD87478",
    #"WD87478",
    #"WD85806",
    #"WD85806",
    #"WD87480",
    #"WD87480",
    #"WD87482",
    #"WD87482",
    #"MPP_00274_A1",
    #"MPP_00274_B1",
    #"MPP_00092_A1",
    #"vTMA_001",
    #"vTMA_002",
    #"WD84221",
    #"HTA11_07862",
    #"HTA11_10711",
    #"HTA11_07663",
    #"HTA11_06134",
    #"WD84602",
    #"WD84602",
    #"WD84596",
    #"WD84594",
    #"WD84226",
    #"WD84216",
    #"WD84216",
    "vTMA_003_004_005",
    "vTMA_003_004_005",
    "vTMA_003_004_005",
    #"SR00001",
]
visium_IDs = [
    #"6723_1",
    #"6723_2",
    #"6723_3",
    #"6723_4",
    #"7003_1",
    #"7003_2",
    #"7003_3",
    #"7003_4",
    #"7003_5",
    #"7003_6",
    #"7003_7",
    #"7003_8",
    #"7319_2",
    #"7319_3",
    #"7319_4",
    #"7794_1",
    #"7794_2",
    #"7794_3",
    #"8270_1",
    #"8270_2",
    #"8270_3",
    #"8270_5",
    #"8270_6",
    #"8270_7",
    #"8270_8",
    #"8270_9",
    #"8270_10",
    #"8270_11",
    #"8270_12",
    "8578_1",
    "8578_2",
    "8578_3",
    #"8578_4",
]
flips = [
    #False,
    #False,
    #False,
    #True,
    #True,
    #True,
    #True,
    #False,
    #True,
    #False,
    #False,
    #False,
    #True,
    #False,
    #True,
    #True,
    #False,
    #False,
    #False,
    #True,
    #False,
    #False,
    #False,
    #False,
    #True,
    #False,
    #True,
    #True,
    #True,
    True,
    False,
    True,
    #False,
]

for block, visium_ID, flip in zip(blocks, visium_IDs, flips):
    affine = np.genfromtxt("/mnt/500gb_ssd/home/cody/images/MxIF/{}/{}_downsample8_{}_affine.csv".format(block, block, visium_ID), delimiter=",")
    shape = np.genfromtxt("/mnt/500gb_ssd/home/cody/images/MxIF/{}/{}_downsample8_{}_shape.csv".format(block, block, visium_ID), delimiter=",")
    
    print("Transforming {}, {}".format(block, visium_ID))
    transform_imgs(
        affine=affine,
        shape=shape,
        img_in="/mnt/500gb_ssd/home/cody/images/MxIF/{}/{}_downsample8.npz".format(block, block),
        img_out="/mnt/500gb_ssd/home/cody/images/MxIF/{}/{}_downsample8_{}.npz".format(block, block, visium_ID),
        flip=flip,
        downsample_fact=None,
        plot=True,
    )